# NASA Mars News (Web Scraping /Browser Automation using Splinter)



<img src="images/mission_to_mars.jpg">



#### Python Packages : <ol> <li> Splinter</li> <li> BeautifulSoup</li> <li> Pandas</li>
    

In [87]:
from splinter import Browser
from bs4 import BeautifulSoup

In [88]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)


In [91]:
def scrape():
    
    listings = {}#Dictionary to store all the reqired informations. 
    
    
    #Automating browser to collect the latest News Title and Paragraph Text.
    browser = init_browser()
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    listings["latest_title"]= soup.find('div',class_='content_title').get_text()
    listings["description"]=soup.find('div',class_="article_teaser_body").text
    browser.quit()
    
    
    #Automating browser to find the image url for the current Featured Mars Image.
    browser = init_browser()
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    listings["featured_image_url"]="https://www.jpl.nasa.gov"+soup.find('div', class_="carousel_items").a['data-fancybox-href']
    browser.quit()
    
    
    #Automating browser to collect the latest Mars weather.
    browser = init_browser()
    url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    listings["mars_weather"]=soup.find('div',class_="ProfileTimeline").p.text
    browser.quit()
    
    
    #Using Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
    import pandas as pd
    url = 'https://space-facts.com/mars/'
    tables = pd.read_html(url)
    df = tables[0]
    df.columns = ['Description','Value']
    df.set_index('Description' ,inplace=True)
    listings["html_table"] = df.to_html()
    browser.quit()
    
    
    
    #Automating browser to obtain high resolution images for each of Mar's hemispheres.
    browser = init_browser()
    url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    results=soup.find_all('div',class_='item')
    title=[] # List to collect the names of the Mar's hemispheres
    img_uurl=[] #List to collect the img urls of each of the Mar's hemispheres
    for i in results:
        text=i.h3.text
        title.append(i.h3.text)
        browser.click_link_by_partial_text(text)
        current_page_html=browser.html
        soup = BeautifulSoup(current_page_html, 'lxml')
        url=soup.find_all('img',class_='wide-image')[0]
        img_url.append('https://astrogeology.usgs.gov'+url['src'])
        browser.back()
    

    hemisphere_image_urls=[]
    for i in range(4):
        hemisphere_image_urls.append({
        'img_url':img_url[i],
        'title':title[i],
            })

    listings["hemisphere_image_urls"]= hemisphere_image_urls
    browser.quit()
    return listings

In [92]:
scrape()

{'description': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19639_ip.jpg',
 'hemisphere_image_urls': [{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'html_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\